In [1]:
import numpy as numpy
import networkx as nx
import scipy as sp
import pickle
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy import sparse
from collections import defaultdict
from scipy.sparse import csr_matrix, spmatrix
import numpy as np
from sklearn.metrics import *
from scipy import spatial
from scipy.sparse import csr_matrix
import scipy.sparse
import scipy as sp
import tqdm as tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
def dotSimilarity(fp, vec):
    ''' gets similarity between a fingerprint and a row vector
        the number of non-zero components they share 
        divided by the total number of non-zero components of the vector '''
    return vec.dot(fp).max() / vec.sum()

In [3]:
def cosineSimilarity(fp,vec):
    if scipy.sparse.isspmatrix_csr(vec):
        return 1 - spatial.distance.cosine(fp, vec.A.astype(np.float))
    else:
        return 1 - spatial.distance.cosine(fp, vec)

In [4]:
def NMISimilarity(fp,vec):
    if scipy.sparse.isspmatrix_csr(vec):       
        return normalized_mutual_info_score(fp,vec.A.astype(np.float).flatten())
    else:
        return normalized_mutual_info_score(fp, vec)

In [5]:
def updateFingerprint(fp, vec, count):
    ''' updates a fingerprint when a node vector is added to the cluster
        weighted merge of the node vector with the fingerprint '''
    if scipy.sparse.isspmatrix_csr(vec):
        return (fp * ((count-1)/count)) + (vec.A.astype(np.float) * (1/count))
    else:
        return (fp * ((count-1)/count)) + (vec*(1/count))

In [6]:
def findClusters(nodes, csr_matrix, similarity='dotsim', threshold=0.5):
    ''' fingerprint map
    {
        fp_index: [
            row_index,
            ...
        ],
        ...
    }
    '''
    # mapping of nodes to fingerprints to keep track of what node belongs to what fp
    fmap = defaultdict(list)

    ''' fingerprints '''
    fps = []

    for ri, node in enumerate(nodes):
#         print(ri,node)
        row = csr_matrix[ri]
        
        # initialize fingerprints
        if len(fps) == 0:
            fmap[len(fps)].append(node)
            fps.append(row.A[0].astype(np.float))
            continue
        
        # get best scoring fingerprint using dotSimilarity
        if similarity == 'dotsim':
            print('initial similarity metric: dot_similarity')
            
            # sorted and pop gets me the best scoring one (find something more elegant!)
            score, fi, fp = sorted([(dotSimilarity(fp, row), fi, fp) for fi, fp in enumerate(fps)]).pop() 

        # get best scoring fingerprint using cosine Similarity
        elif similarity == 'cosine':
            print('initial similarity metric: cosine_similarity')
            score, fi, fp = sorted([(cosineSimilarity(fp, row), fi, fp) for fi, fp in enumerate(fps)]).pop() 
        
        # get best scoring fingerprint using mutual_info_score
        elif similarity == 'nmi':
            print('initial similarity metric: normalized_mutual_information')
            score, fi, fp = sorted([(NMISimilarity(fp, row), fi, fp) for fi, fp in enumerate(fps)]).pop()  

        if score > threshold:
            print(score, threshold)
            
            # map node to fingerprint
            fmap[fi].append(node)
            # update fingerprint with row weights
            fp[:] = updateFingerprint(fp, row, len(fmap[fi]))
            
        else:
            fmap[len(fps)].append(node)
            fps.append(row.A[0].astype(np.float))
            
    return fps, fmap

In [7]:
def mergeFingerprints(fps, fmap, similarity='dotsim', threshold=0.3):
    '''
    finds similar clusters and merges them  
    '''
    # same kind of mapping of nodes to fingerprints
    merged_fps = []
    merged_fmap = {}

    processed = []
    for ai, afp in enumerate(fps):
        # skip fingerprints that have already been merged
        if ai in processed: continue
            
        # dot similarity
        if similarity == 'dotsim':
            print('merging similarity metric: dot_similarity')
            score, bi, bfp = sorted([(dotSimilarity(afp, bfp), bi, bfp) for bi, bfp in enumerate(fps) if bi != ai]).pop()

        # normalized mutual info score
        elif similarity == 'nmi':
            print('initial similarity metric: normalized_mutual_information')
            score, bi, bfp = sorted([(NMISimilarity(afp, bfp), bi, bfp) for bi, bfp in enumerate(fps) if bi != ai]).pop()

        
        # cosine Similarity
        elif similarity == 'cosine':
            print('initial similarity metric: cosine_similarity')
            score, bi, bfp = sorted([(cosineSimilarity(afp, bfp), bi, bfp) for bi, bfp in enumerate(fps) if bi != ai]).pop() 
                      
        # same for second fingerprint
        if bi in processed: continue

        if score > threshold:
            # merge fingerprints
            fp = updateFingerprint(afp, bfp, 2)
            merged_fps.append(fp)
            # merge node references
            i = len(merged_fps) - 1
            merged_fmap[i] = list(set(fmap[ai] + fmap[bi]))
            # mark as processed
            processed += [ai, bi]
            
    # add fingerprints that were not merged
    for i, fp in enumerate(fps):
        if i not in processed:
            merged_fps.append(fp)
            merged_fmap[len(merged_fps) - 1] = fmap[i]

    return merged_fps, merged_fmap

In [11]:
# def mergeFingerprints(fps, fmap, threshold=0.3):
#     '''
#     finds similar clusters and merges them  
#     '''
#     # same kind of mapping of nodes to fingerprints
#     merged_fps = []
#     merged_fmap = {}

#     processed = []
#     for ai, afp in enumerate(fps):
#         # skip fingerprints that have already been merged
#         if ai in processed: continue

#         # get best scoring fingerprint using dotSimilarity
# #         score, bi, bfp = sorted([(dotSimilarity(afp, bfp), bi, bfp) for bi, bfp in enumerate(fps) if bi != ai]).pop()

#         # get best scoring fingerprint using mutual_info_score
#         # normalized_mutual_info_score(x,y) 
#         # adjusted_mutual_info_score(x,y)
#         score, bi, bfp = sorted([(normalized_mutual_info_score(afp, bfp), bi, bfp) for bi, bfp in enumerate(fps) if bi != ai]).pop()

#         # same for second fingerprint
#         if bi in processed: continue

#         if score > threshold:
#             # merge fingerprints
#             fp = updateFingerprint(afp, bfp, 2)
#             merged_fps.append(fp)
#             # merge node references
#             i = len(merged_fps) - 1
#             merged_fmap[i] = list(set(fmap[ai] + fmap[bi]))
#             # mark as processed
#             processed += [ai, bi]
            
#     # add fingerprints that were not merged
#     for i, fp in enumerate(fps):
#         if i not in processed:
#             merged_fps.append(fp)
#             merged_fmap[len(merged_fps) - 1] = fmap[i]

#     return merged_fps, merged_fmap

In [8]:
trajectory = np.load('../sample_data/trajectory.npy')
t = np.matrix(trajectory)
csr_trajectory = sparse.csr_matrix(trajectory)

In [ ]:
# choose thresholds
first_th = 0.7
second_th = 0.9

# find initial clusters
# findClusters returns fps (list) and fmap (dict)
fps, fmap = findClusters(rows, t, first_th)
print('clusters found: ' + str(len(fmap)))

# merge similar clusters
merged_fps, merged_fmap = mergeFingerprints(fps, fmap, second_th)
print('clusters merged: ' + str(len(fmap)-len(merged_fmap)))
print('remaining clusters: ' + str(len(merged_fmap)))

0.9048233444321127 0.9
0.9009808866913572 0.9
0.9006851586309836 0.9
0.9006709689565728 0.9
0.9006546599807018 0.9
0.9006829834407262 0.9
0.9006891956037538 0.9
0.9006439176737459 0.9
0.9009380642260487 0.9
0.9032759360190811 0.9
0.9003231613767171 0.9
0.9027970822339201 0.9
0.9007281186742803 0.9
0.9010465327398122 0.9
0.900194638638703 0.9
clusters found: 9986


### USING THIS CODE

In [11]:
from reader import *
from mpi4py import MPI

file = '../sample_data/test.tsv'
g = nx.read_edgelist(file, delimiter=' ', nodetype=int, edgetype=int, create_using=nx.Graph())
nodes = g.nodes()

gr = GraphReader('../sample_data/test.tsv', '../sample_data/test_node_edges.txt')
csr_matrix = gr.read() # sparse matrix

print('nodes: ' + str(len(g.nodes)))
csr_A = nx.to_scipy_sparse_matrix(g, format='csr', dtype=np.float64)

nodes: 12


In [88]:
csr_matrix

<12x12 sparse matrix of type '<class 'numpy.float64'>'
	with 50 stored elements in Compressed Sparse Row format>

In [89]:
csr_A

<12x12 sparse matrix of type '<class 'numpy.float64'>'
	with 50 stored elements in Compressed Sparse Row format>

In [90]:
csr_matrix.todense() == csr_A.todense()

matrix([[ True,  True,  True,  True,  True, False,  True, False,  True,
          True,  True,  True],
        [ True,  True,  True,  True,  True, False,  True, False,  True,
          True,  True,  True],
        [ True,  True,  True,  True,  True, False,  True, False,  True,
          True,  True,  True],
        [ True,  True,  True,  True,  True, False,  True, False,  True,
          True,  True,  True],
        [ True,  True,  True,  True,  True,  True, False, False,  True,
          True,  True,  True],
        [False, False, False, False,  True,  True, False,  True,  True,
         False, False, False],
        [ True,  True,  True,  True, False, False,  True, False, False,
          True, False, False],
        [False, False, False, False, False,  True, False,  True,  True,
         False, False,  True],
        [ True,  True,  True,  True,  True,  True, False,  True,  True,
          True, False,  True],
        [ True,  True,  True,  True,  True, False,  True, False,  True,
 

In [91]:
csr_matrix.todense()

matrix([[0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
        [1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1.],
        [1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.]])

In [92]:
csr_A.todense()

matrix([[0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [10]:
# choose thresholds
first_th = 0.222
second_th = 0.5

# find initial clusters
# findClusters returns fps (list) and fmap (dict)
fps, fmap = findClusters(nodes, csr_matrix, similarity='nmi', threshold=first_th)
print('clusters found: ' + str(len(fmap)))

# merge similar clusters
# merge similar clusters
merged_fps, merged_fmap = mergeFingerprints(fps, fmap, similarity='nmi', threshold=second_th)
print('clusters merged: ' + str(len(fmap)-len(merged_fmap)))
print('remaining clusters: ' + str(len(merged_fmap)))

initial similarity metric: normalized_mutual_information
0.3407833215844391 0.222
initial similarity metric: normalized_mutual_information
0.5817448669547886 0.222
initial similarity metric: normalized_mutual_information
0.6051084728780068 0.222
initial similarity metric: normalized_mutual_information
0.4134132143817069 0.222
initial similarity metric: normalized_mutual_information
initial similarity metric: normalized_mutual_information
0.4000540705765895 0.222
initial similarity metric: normalized_mutual_information
0.5456278264547397 0.222
initial similarity metric: normalized_mutual_information
0.3500037108861601 0.222
initial similarity metric: normalized_mutual_information
0.391623268955809 0.222
initial similarity metric: normalized_mutual_information
0.391623268955809 0.222
initial similarity metric: normalized_mutual_information
0.6254165429705678 0.222
clusters found: 2
initial similarity metric: normalized_mutual_information
initial similarity metric: normalized_mutual_infor

In [64]:
np.array([[1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0]]).flatten()

array([1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0])

In [9]:
# choose thresholds
first_th = 0.222
second_th = 0.05

# find initial clusters
# findClusters returns fps (list) and fmap (dict)
fps, fmap = findClusters(nodes, csr_matrix, similarity='dotsim', first_th)
print('clusters found: ' + str(len(fmap)))

# merge similar clusters
# merge similar clusters
merged_fps, merged_fmap = mergeFingerprints(fps, fmap, second_th)
print('clusters merged: ' + str(len(fmap)-len(merged_fmap)))
print('remaining clusters: ' + str(len(merged_fmap)))

SyntaxError: positional argument follows keyword argument (<ipython-input-9-efac57266dc8>, line 7)

In [3]:
# running this cell will load the functions from the clusters file 
from clusters import *

# you need a networkx graph, this is a small one for testing
file = '../sample_data/zebra.tsv'
g = nx.read_edgelist(file, delimiter=' ', nodetype=int, create_using=nx.Graph())

print(nx.info(g))
print('Density: ' + str(nx.density(g)))

# choose thresholds
first_th = 0.222
second_th = 0.05

# find initial clusters
# findClusters returns fps (list) and fmap (dict)
fps, fmap = findClusters(g, first_th)
print('clusters found: ' + str(len(fmap)))

# merge similar clusters
merged_fps, merged_fmap = mergeFingerprints(fps, fmap, second_th)
print('clusters merged: ' + str(len(fmap)-len(merged_fmap)))
print('remaining clusters: ' + str(len(merged_fmap)))

Name: 
Type: Graph
Number of nodes: 27
Number of edges: 111
Average degree:   8.2222
Density: 0.3162393162393162


TypeError: 'float' object is not subscriptable